In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
import pandas as pd
import numpy as np
import torch
import datetime
from torch import nn
from torchvision import transforms
from cnn_finetune import make_model

import model as M
from dataset import SegmentationDataset
from data import *
from util import *

In [3]:
torch.cuda.is_available()
device = torch.device('cuda')

# Setup dataset

In [4]:
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)
depth_df = pd.read_csv(depth_csv)

train_df = pd.merge(train_df, depth_df, on='id')
test_df = pd.merge(test_df, depth_df, on='id')

In [5]:
train_dataset = SegmentationDataset(train_df, mode='train').set_transformer((128, 128))
test_dataset = SegmentationDataset(test_df, mode='test').set_transformer((128, 128))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

# Training

In [13]:
torch.cuda.empty_cache()

model = M.UNet(1, n_classes=1)
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

In [14]:
def train(model, n_epoch, train_iter):
    model.train()
    
    for epoch in range(n_epoch):
        total_loss = 0
        total_size = 0
        
        for batch_idx, (data, target) in enumerate(train_iter):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            
            # Forward
            output = model(data)
            loss = criterion(output.view(-1), target.view(-1))
            
            total_loss += loss.item()
            total_size += data.size(0)
            
            # Backward
            loss.backward()
            optimizer.step()
            
            if batch_idx % 100 == 0:
                now = datetime.datetime.now()
                print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tAverage loss: {:.6f}'.format(
                    now,
                    epoch, batch_idx * len(data), len(train_iter.dataset),
                    100. * batch_idx / len(train_iter), total_loss / total_size))
    return model

In [17]:
model = train(model, 2, train_loader)

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


[2018-09-25 21:46:46.164246] Train Epoch: 0 [0/4000 (0%)]	Average loss: 0.046805
[2018-09-25 21:47:25.335303] Train Epoch: 0 [1600/4000 (40%)]	Average loss: 0.032115
[2018-09-25 21:48:04.552408] Train Epoch: 0 [3200/4000 (80%)]	Average loss: 0.029004
[2018-09-25 21:48:25.668434] Train Epoch: 1 [0/4000 (0%)]	Average loss: 0.032603
[2018-09-25 21:49:04.941412] Train Epoch: 1 [1600/4000 (40%)]	Average loss: 0.024940
[2018-09-25 21:49:44.177444] Train Epoch: 1 [3200/4000 (80%)]	Average loss: 0.024603


In [30]:
torch.empty(3).random_(2)

tensor([0., 1., 1.])

In [8]:
torch.cuda.empty_cache()
for x, t in train_loader:
    x = x.to(device)
    t = t.to(device)
    
    y = model(x)
    break

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


In [9]:
y.view(-1).shape

torch.Size([262144])

In [10]:
t.view(-1).shape

torch.Size([262144])

In [12]:
t

tensor([[[[0., 0., 0.,  ..., 1., 1., 1.],
          [0., 0., 0.,  ..., 1., 1., 1.],
          [0., 0., 0.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 0., 0., 0.],
          [1., 1., 1.,  ..., 0., 0., 0.],
          [1., 1., 1.,  ..., 0., 0., 0.],
          ...,
          [1., 1., 1.,  ..., 0., 0., 0.],
          [1., 1., 1.,  ..., 0., 0., 0.],
          [1., 1., 1.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0.